# Практическая работа

# Задача

Один из способов повысить эффективность взаимодействия банка с клиентами — отправлять предложение о новой услуге не всем клиентам, а только некоторым, которые выбираются по принципу наибольшей склонности к отклику на это предложение.

Задача заключается в том, чтобы предложить алгоритм, который будет выдавать склонность клиента к положительному или отрицательному отклику на предложение банка. Предполагается, что, получив такие оценки для некоторого множества клиентов, банк обратится с предложением только к тем, от кого ожидается положительный отклик.


Для решения этой задачи загрузите файлы из базы в Postgres.
Эта БД хранит информацию о клиентах банка и их персональные данные, такие как пол, количество детей и другие.

Описание таблиц с данными представлено ниже.


**D_work**

Описание статусов относительно работы:
- ID — идентификатор социального статуса клиента относительно работы;
- COMMENT — расшифровка статуса.


**D_pens**

Описание статусов относительно пенсии:
- ID — идентификатор социального статуса;
- COMMENT — расшифровка статуса.


**D_clients**

Описание данных клиентов:
- ID — идентификатор записи;
- AGE	— возраст клиента;
- GENDER — пол клиента (1 — мужчина, 0 — женщина);
- EDUCATION — образование;
- MARITAL_STATUS — семейное положение;
- CHILD_TOTAL	— количество детей клиента;
- DEPENDANTS — количество иждивенцев клиента;
- SOCSTATUS_WORK_FL	— социальный статус клиента относительно работы (1 — работает, 0 — не работает);
- SOCSTATUS_PENS_FL	— социальный статус клиента относительно пенсии (1 — пенсионер, 0 — не пенсионер);
- REG_ADDRESS_PROVINCE — область регистрации клиента;
- FACT_ADDRESS_PROVINCE — область фактического пребывания клиента;
- POSTAL_ADDRESS_PROVINCE — почтовый адрес области;
- FL_PRESENCE_FL — наличие в собственности квартиры (1 — есть, 0 — нет);
- OWN_AUTO — количество автомобилей в собственности.


**D_agreement**

Таблица с зафиксированными откликами клиентов на предложения банка:
- AGREEMENT_RK — уникальный идентификатор объекта в выборке;
- ID_CLIENT — идентификатор клиента;
- TARGET — целевая переменная: отклик на маркетинговую кампанию (1 — отклик был зарегистрирован, 0 — отклика не было).
    
    
**D_job**

Описание информации о работе клиентов:
- GEN_INDUSTRY — отрасль работы клиента;
- GEN_TITLE — должность;
- JOB_DIR — направление деятельности внутри компании;
- WORK_TIME — время работы на текущем месте (в месяцах);
- ID_CLIENT — идентификатор клиента.


**D_salary**

Описание информации о заработной плате клиентов:
- ID_CLIENT — идентификатор клиента;
- FAMILY_INCOME — семейный доход (несколько категорий);
- PERSONAL_INCOME — личный доход клиента (в рублях).


**D_last_credit**

Информация о последнем займе клиента:
- ID_CLIENT — идентификатор клиента;
- CREDIT — сумма последнего кредита клиента (в рублях);
- TERM — срок кредита;
- FST_PAYMENT — первоначальный взнос (в рублях).


**D_loan**

Информация о кредитной истории клиента:
- ID_CLIENT — идентификатор клиента;
- ID_LOAN — идентификатор кредита.

**D_close_loan**

Информация о статусах кредита (ссуд):
- ID_LOAN — идентификатор кредита;
- CLOSED_FL — текущий статус кредита (1 — закрыт, 0 — не закрыт).

Ниже представлен минимальный список колонок, которые должны находиться в итоговом датасете после склейки и агрегации данных. По своему усмотрению вы можете добавить дополнительные к этим колонки.

    - AGREEMENT_RK — уникальный идентификатор объекта в выборке;
    - TARGET — целевая переменная: отклик на маркетинговую кампанию (1 — отклик был зарегистрирован, 0 — отклика не было);
    - AGE — возраст клиента;
    - SOCSTATUS_WORK_FL — социальный статус клиента относительно работы (1 — работает, 0 — не работает);
    - SOCSTATUS_PENS_FL — социальный статус клиента относительно пенсии (1 — пенсионер, 0 — не пенсионер);
    - GENDER — пол клиента (1 — мужчина, 0 — женщина);
    - CHILD_TOTAL — количество детей клиента;
    - DEPENDANTS — количество иждивенцев клиента;
    - PERSONAL_INCOME — личный доход клиента (в рублях);
    - LOAN_NUM_TOTAL — количество ссуд клиента;
    - LOAN_NUM_CLOSED — количество погашенных ссуд клиента.


Будьте внимательны при сборке датасета: это реальные банковские данные, в которых могут наблюдаться дубли, некорректно заполненные значения или значения, противоречащие друг другу. Для получения качественной модели необходимо предварительно очистить датасет от такой информации.

## Задание 1

В предыдущем задании вы собрали всю информацию о клиентах в одну таблицу, где одна строчка соответствует полной информации об одном клиенте.

Загрузите эту таблицу.

In [101]:
import streamlit as st
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

import warnings

warnings.filterwarnings('ignore')

In [102]:
df = pd.read_csv('df_prepare.csv')

In [103]:
numerical = ['AGE',
             'WORK_TIME',
             'PERSONAL_INCOME',
             'CREDIT',
             'TERM',
             'FST_PAYMENT']

id_list = ['AGREEMENT_RK',
           'ID_CLIENT',
           'TARGET',
           'ID_LOAN']

cat = list(df.columns)
cat = list(set(cat) - set(numerical) - set(id_list))

In [104]:
#Закодируем категориальные фичи
df = pd.get_dummies(df, columns=cat)

In [105]:
df.to_csv('./df_model.csv', index=False)

Разбейте данные на тренировочную и тестовую часть в пропорции 80% к 20%, зафиксируйте `random_state = 42`.

In [106]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(['TARGET', 'AGREEMENT_RK', 'ID_CLIENT', 'ID_LOAN'], axis=1), df['TARGET'], test_size=0.2, random_state=42)

На тренировочных данных обучите линейную модель классификации для предсказания целевой переменной (столбец `TARGET`).

Сделайте прогноз вероятности отклика на рекламную кампанию для тестовых данных.

In [107]:
model_base = LogisticRegression().fit(X_train, y_train)
pred = model_base.predict(X_test)
pred_prob = model_base.predict_proba(X_test)
pred_prob_train = model_base.predict_proba(X_train)

Переведите вероятности в классы по стандартному порогу (0.5) и на тестовом наборе данных вычислите метрики:

* accuracy
* precision
* recall
* f1-score

In [108]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [109]:
predict = np.where(pred_prob[:, 1] < 0.5, 0, 1)

In [110]:
def metrics(y, predict):
    accur = accuracy_score(y, predict)
    prec = precision_score(y, predict)
    rec = recall_score(y, predict)
    f1 = f1_score(y, predict)

    print (f"accuracy: {accur}\nprecision: {prec}\nrecall: {rec}\nf1-score: {f1}")

In [111]:
metrics(y_test, predict)

accuracy: 0.8760330578512396
precision: 0.5
recall: 0.01875
f1-score: 0.03614457831325301


Целевая метрика для задачи - полнота, так как нам нужно найти максимум клиентов, кто может откликнуться на рекламу.

Но при этом точность не должна просесть, поэтому за ней тоже следим.

Разбейте тренировочные данные на `train` и `val` части в пропорции 3 к 1.

В цикле:

* переберите пороги от 0 до 1 с шагом 0.01
* вычислите для каждого порога значение метрик precision и recall
* подберите такой порог, при котором recall не меньше 0.66, а точность максимальна.

In [112]:
thr = int(pred_prob_train.shape[0]-pred_prob_train.shape[0]/3)

In [113]:
prob_train = pred_prob_train[:thr]
prob_val = pred_prob_train[thr:]

y_train_= y_train[:thr]
y_val= y_train[thr:]

In [114]:
from tqdm import tqdm
res = []
for i in tqdm(np.arange(0, 1, 0.01)):    
    temp_pred = np.where(prob_train[:, 1] < i, 0, 1)
    res.append([i, precision_score(y_train_, temp_pred), recall_score(y_train_, temp_pred), accuracy_score(y_train_, temp_pred)])

res_train = pd.DataFrame(res, columns=['thr', 'precision', 'recall', 'accuracy'])

100%|██████████| 100/100 [00:00<00:00, 317.10it/s]


In [115]:
res_train[res_train['recall'] >= 0.66].sort_values(by='accuracy', ascending=False)

,thr,precision,recall,accuracy
11,0.11,0.187875,0.714751,0.598896
10,0.10,0.180377,0.779136,0.552983
9,0.09,0.168352,0.814181,0.499903
8,0.08,0.158918,0.852486,0.446242
7,0.07,0.150704,0.889976,0.390837
6,0.06,0.142713,0.924205,0.331170
5,0.05,0.135748,0.955990,0.271406
4,0.04,0.128907,0.974735,0.214161
3,0.03,0.124527,0.991850,0.170283
2,0.02,0.120696,1.000000,0.134153


In [116]:
metrics(y_val, np.where(prob_val[:, 1] < 0.11, 0, 1))

accuracy: 0.6050745690490025
precision: 0.1912212081703607
recall: 0.7119741100323624
f1-score: 0.3014731072285029


Для выбранного порога посчитайте все метрики на тестовых данных. Сильно ли они отличаются от метрик на валидации?

In [117]:
metrics(y_test, np.where(pred_prob[:, 1] < 0.11, 0, 1))

accuracy: 0.5971074380165289
precision: 0.18160377358490565
recall: 0.6416666666666667
f1-score: 0.28308823529411764


посчитанные метрики по валидации и тесту отличаются, естественно не в лучшую сторону для теста. тк модель на этих данных не училась

Выведите на экран в виде таблицы топ-6 признаков с наибольшими по модулю весами модели.

In [118]:
pd.DataFrame(abs(model_base.coef_.T), index=X_train.columns, columns=['weight_abs']).sort_values(by='weight_abs', ascending=False).head(6)

,weight_abs
PERSONAL_INCOME,1.583226
POSTAL_ADDRESS_PROVINCE_Москва,1.367107
GEN_INDUSTRY_Недвижимость,1.215771
CHILD_TOTAL_5,1.197729
WORK_TIME,1.052504
FST_PAYMENT,1.040474


In [119]:
#Для нашего приложения стримлит сохраним  модель

import pickle

with open('model.pickle', 'wb') as f:
    pickle.dump(model_base, f)

## Задание 2

Добавьте в Streamlit-приложение визуализацию результатов модели:

* опцию выбора порога и вывод метрик качества в зависимости от выбранного порога

* вывод прогноза модели на выбранном объекте (клиенте) - вероятность отклика на рекламу.

## Бонус

Попробуйте применить другие модели классификации для решения этой задачи (любые какие знаете).

Удалось ли добиться улучшения качества модели?

In [120]:
# т.к. я леопард, то конечно же кэтбуст
from catboost import CatBoostClassifier

In [121]:
model = CatBoostClassifier(eval_metric='AUC')

In [122]:
#т.к это кэтбуст то тут кодировать категориальные фичи не нужно, и нормировка не дает никаого результата, поэтому загрузим датасет без вот этого всего
df_cat = pd.read_csv('df.csv')
X_train, X_test, y_train, y_test = train_test_split(df_cat.drop(['TARGET', 'AGREEMENT_RK', 'ID_CLIENT', 'ID_LOAN'], axis=1), df_cat['TARGET'], test_size=0.2, random_state=42)

In [123]:
model.fit(X_train, y_train, eval_set=(X_test, y_test), plot=True, verbose=False, cat_features=cat)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [124]:
#Повторим тот же трюк с выбором порога для чистоты эксперимента
pred_proba_cat = model.predict_proba(X_test)

res = []
for i in tqdm(np.arange(0, 1, 0.01)):    
    temp_pred = np.where(pred_proba_cat[:, 1] < i, 0, 1)
    res.append([i, precision_score(y_test, temp_pred), recall_score(y_test, temp_pred), accuracy_score(y_test, temp_pred)])

res = pd.DataFrame(res, columns=['thr', 'precision', 'recall', 'accuracy'])
res[res['recall'] >= 0.66].sort_values(by='accuracy', ascending=False)

  0%|          | 0/100 [00:00<?, ?it/s]

100%|██████████| 100/100 [00:00<00:00, 604.83it/s]


,thr,precision,recall,accuracy
12,0.12,0.295247,0.672917,0.760331
11,0.11,0.273539,0.702083,0.731921
10,0.10,0.252323,0.735417,0.697056
9,0.09,0.233609,0.764583,0.659866
8,0.08,0.218732,0.812500,0.616994
7,0.07,0.206397,0.860417,0.572572
6,0.06,0.185445,0.881250,0.505424
5,0.05,0.170825,0.927083,0.433110
4,0.04,0.155646,0.956250,0.351498
3,0.03,0.142944,0.977083,0.270919


In [125]:
#Выведим метрики по старому порогу 0.11
metrics(y_test, np.where(pred_proba_cat[:, 1] < 0.11, 0, 1))

accuracy: 0.7319214876033058
precision: 0.273538961038961
recall: 0.7020833333333333
f1-score: 0.3936915887850467


Конечно же результат у кэтбуста лучше, чем у линейной регрессии, и все это с базовыми параметрами на рок-ауке